In [1]:
import findspark
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import col, year, month, max, min, mean, count, desc, asc, avg, round, countDistinct, sum, quarter, format_number, concat, lit, lag, when

import matplotlib.pyplot as plt 
import seaborn as sns 

In [2]:
findspark.init()

spark = SparkSession.builder \
    .appName("Data Analysis") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.extraClassPath", r"C:\Drivers\sqljdbc_12.10.0.0_enu\sqljdbc_12.10\enu\jars\mssql-jdbc-12.10.0.jre11.jar") \
    .getOrCreate()

# Read CSV
customers = spark.read.csv(
    "../final data/customer_features.csv",
    header=True,
    inferSchema=True,
    sep=",",
    quote='"',
    escape='"',
    multiLine=True  
)

orders = spark.read.csv(
    "../final data/orders_facts.csv",
    header=True, 
    inferSchema=True, 
    sep=",",
    quote='"',
    escape='"',
    multiLine=True
)


# Get to Know Your Customers First: Build Complete Customer Profiles

In [3]:
customers.show()

+-----------+-------------------+------------------+---------------+---------------------+-----------------------+----------------------------+-----------------------+------------------------+----------------------+------------------+--------------------------+--------------------------------+---------------------+---------------------+----------------------------------+--------------------------+----------------+-----------------------+---------------------+-----------------+--------------------------+--------------------------+---------------------------+
|customer_id|customer_first_date|customer_last_date|customer_tenure|customer_no_of_orders|customer_total_quantity|customer_average_order_value|customer_historical_clv|customer_diversity_score|customer_total_revenue|customer_is_repeat|customer_churn_probability|customer_avg_days_between_orders|customer_return_count|customer_return_value|customer_first_order_product_count|customer_first_order_value|customer_recency|customer_shipment_c

In [4]:
customers = customers \
    .withColumn("first_year", year(col("customer_first_date"))) \
    .withColumn("first_month", month(col("customer_first_date")))


#### How many new customers am I acquiring each  year and month?

In [5]:
customers.select("customer_id", "first_year") \
    .groupBy("first_year") \
    .agg(count("customer_id").alias("customer_count")) \
    .orderBy(asc("first_year")) \
    .show()

+----------+--------------+
|first_year|customer_count|
+----------+--------------+
|      2010|          1130|
|      2011|           714|
|      2012|           462|
|      2013|           397|
+----------+--------------+



In [6]:
customers.select("customer_id", "first_year", "first_month") \
    .groupBy("first_year", "first_month") \
    .agg(count("customer_id").alias("customer_count")) \
    .orderBy(asc("first_year"), asc("first_month")) \
    .show()

+----------+-----------+--------------+
|first_year|first_month|customer_count|
+----------+-----------+--------------+
|      2010|          1|           141|
|      2010|          2|            96|
|      2010|          3|           125|
|      2010|          4|           105|
|      2010|          5|           103|
|      2010|          6|            84|
|      2010|          7|            92|
|      2010|          8|            89|
|      2010|          9|            74|
|      2010|         10|            77|
|      2010|         11|            74|
|      2010|         12|            70|
|      2011|          1|            74|
|      2011|          2|            64|
|      2011|          3|            62|
|      2011|          4|            53|
|      2011|          5|            74|
|      2011|          6|            60|
|      2011|          7|            56|
|      2011|          8|            49|
+----------+-----------+--------------+
only showing top 20 rows



# What is our true cost ot acquire new customers? 

Since we don't have the marketing and expenses data we cannot get the answer to that, however for now we are losing money in discounts trying to acquire customers and retain customers so we might as well use the discount to know how much on average we are losing because of that.

In [7]:
orders\
.join(
    customers, "customer_id"
).withColumn(
    "order_year", year("order_date")
).filter(
    col("order_year") == col("first_year")
).select(
    "customer_id", 
    "discount_amount", 
    "first_year"
).groupBy(
    "first_year"
).agg(
    round(sum("discount_amount"), 2).alias("total_discount_spend"), 
    countDistinct("customer_id").alias("new_customers"), 
    round((sum("discount_amount") / countDistinct("customer_id")), 2).alias("discount_per_customer")
).orderBy(
    asc("first_year")
).show()

+----------+--------------------+-------------+---------------------+
|first_year|total_discount_spend|new_customers|discount_per_customer|
+----------+--------------------+-------------+---------------------+
|      2010|            10230.78|         1130|                 9.05|
|      2011|             4421.12|          714|                 6.19|
|      2012|              2736.7|          462|                 5.92|
|      2013|             3288.64|          397|                 8.28|
+----------+--------------------+-------------+---------------------+



We have the drop in discount per customer in the year 2011 and 2012 possibly due to correct targeting or higher organic growth (less reliance on discounts). A rising discount per customer amount in 2013 can leave us with question of whether their is increasing competition or if discounts were not as effective.

Possible steps 
1. Repeat purchases (Cohort analysis)
* % of discounted customers who made a 2nd purchase 
* If the retition is low, discounts may not be worth it. (how low is low)

####  What is my revenue per customer? How is it trending?

In [8]:
orders\
.join(
    customers, "customer_id"
).select(
    "customer_id", 
    "total_price", 
    "order_date"
).groupBy(
    year(col("order_date"))
).agg(
    round(sum("total_price"), 2).alias("total_revenue"), 
    countDistinct("customer_id").alias("customers"), 
    round((sum("total_price") / countDistinct("customer_id")), 2).alias("total_revenue_per_customer")
).orderBy(
    asc(year(col("order_date")))
).show()

+----------------+-------------+---------+--------------------------+
|year(order_date)|total_revenue|customers|total_revenue_per_customer|
+----------------+-------------+---------+--------------------------+
|            2010|   1890156.31|     1130|                    1672.7|
|            2011|   1921968.12|     1213|                   1584.47|
|            2012|   2209445.78|     1232|                   1793.38|
|            2013|   2828033.35|     1589|                   1779.76|
+----------------+-------------+---------+--------------------------+



Revenue per customer has been increasing for different reasons except for 2011 
* the customer are just increasing their spending with us 
* we acquired new high spending customers 
* we introduced products that were favoured by customers 

#### How seasonal are my revenue and margin?

In [9]:
orders = orders \
.withColumn(
    "order_quarter", quarter(col("order_date")), 
).withColumn(
    "order_year", year(col("order_date"))
).withColumn(
    "order_month", month(col("order_date"))
)

orders.show(2)

+--------------+--------+-----------+----------+---------+------------+----------+----------+--------+----------+---------------+--------------+-------------+--------+-----------+---------+-------------+----------+-----------+
|order_fact_key|order_id|customer_id|product_id|region_id|ship_mode_id|order_date| ship_date|discount|unit_price|discount_amount|discount_price|shipping_cost|quantity|total_price|is_return|order_quarter|order_year|order_month|
+--------------+--------+-----------+----------+---------+------------+----------+----------+--------+----------+---------------+--------------+-------------+--------+-----------+---------+-------------+----------+-----------+
|             1|   88525|          2|       179|        1|           3|2012-05-28|2012-05-30|    0.01|      2.88|           0.03|          2.85|          0.5|       2|        5.7|    false|            2|      2012|          5|
|             2|   88522|          3|       924|     1659|           2|2010-07-07|2010-07-08

In [10]:
quarterly_revenue = orders.select(
    "total_price", 
    "order_quarter", 
    "order_year"
).groupBy(
    "order_year", "order_quarter"
).agg(
    round(sum("total_price"), 2).alias("total_revenue_raw")
).orderBy(
    asc("order_year"), 
    asc("order_quarter")
)

window = Window.orderBy("order_year", "order_quarter")
quarterly_growth = quarterly_revenue.withColumn(
    "prev_quarter_revenue", 
    lag("total_revenue_raw").over(window)
).withColumn(
    "revenue_growth_pct", 
    when(
        col("prev_quarter_revenue").isNull(), 
        lit(0)
    ).otherwise(
        ((col("total_revenue_raw") - col("prev_quarter_revenue")) / col("prev_quarter_revenue")) * 100
    )
)

final_result = quarterly_growth.select(
    "order_year",
    "order_quarter",
    concat(lit("$"), format_number("total_revenue_raw", 2)).alias("total_revenue"),
    concat(
        round("revenue_growth_pct", 2), 
        lit("%")
    ).alias("growth_pct")
).orderBy(
    asc("order_year"), 
    asc("order_quarter")
)

final_result.show()

+----------+-------------+-------------+----------+
|order_year|order_quarter|total_revenue|growth_pct|
+----------+-------------+-------------+----------+
|      2010|            1|  $406,683.52|      0.0%|
|      2010|            2|  $342,842.92|    -15.7%|
|      2010|            3|  $460,035.27|    34.18%|
|      2010|            4|  $680,594.60|    47.94%|
|      2011|            1|  $265,598.30|   -60.98%|
|      2011|            2|  $337,932.41|    27.23%|
|      2011|            3|  $540,158.91|    59.84%|
|      2011|            4|  $778,278.50|    44.08%|
|      2012|            1|  $290,417.35|   -62.68%|
|      2012|            2|  $429,894.96|    48.03%|
|      2012|            3|  $497,909.34|    15.82%|
|      2012|            4|  $991,224.13|    99.08%|
|      2013|            1|  $532,774.77|   -46.25%|
|      2013|            2|  $518,723.23|    -2.64%|
|      2013|            3|  $718,206.97|    38.46%|
|      2013|            4|$1,058,328.38|    47.36%|
+----------+

Strong seasonality exists with Q4 being consistently the strongest quarter and Q1 being consistently the weakest with the exception of 2010. Every year, without fail, **Q4 emerges as the best**, delivering the highest revenue by a wide margin. The numbers don't lie, 2010 Q4: $680K -> 2013 Q4: $1.05 (a 55% increases in just 3 years) mostly likely due to holiday effect. After the Q4 high, Q1 stumbles a universal truth in retail. 2011 Q1: A dismal -61% drop from 2010 Q4. 2013 Q1: Despite being the strongest Q1 ($532K), it still fell -46% from the prior Q4.
This isn’t just a dip it’s a systemic challenge. Customers tighten budgets after splurging in Q4.

#### Is most of my revenue coming from new or repeat buyers

In [18]:
from pyspark.sql.functions import col, sum, asc

# Calculate new customer revenue (first-time purchasers)
new_customers_df = orders.join(
    customers, "customer_id"
).select(
    "customer_id", 
    "first_year", 
    "total_price",
    "order_year"
).filter(
    col("first_year") == col("order_year")
).groupBy(
    "order_year"
).agg(
    sum("total_price").alias("new_customer_revenue"),
    countDistinct("customer_id").alias("new_customer_count")
)

# Calculate returning customer revenue
old_customers_df = orders.join(
    customers, "customer_id"
).select(
    "customer_id", 
    "first_year", 
    "total_price",
    "order_year"
).filter(
    col("first_year") < col("order_year")
).groupBy(
    "order_year"
).agg(
    sum("total_price").alias("returning_customer_revenue"),
    countDistinct("customer_id").alias("returning_customer_count")
)

# Join and calculate metrics
cohort_analysis = new_customers_df.join(
    old_customers_df, "order_year", "outer"
).orderBy(asc("order_year"))

# Calculate percentages and averages
cohort_analysis = cohort_analysis.withColumn(
    "total_revenue",
    col("new_customer_revenue") + col("returning_customer_revenue")
).withColumn(
    "returning_revenue_pct",
    (col("returning_customer_revenue") / col("total_revenue")) * 100
).withColumn(
    "avg_new_customer_value",
    col("new_customer_revenue") / col("new_customer_count")
).withColumn(
    "avg_returning_customer_value",
    col("returning_customer_revenue") / col("returning_customer_count")
)

# Show comprehensive results
cohort_analysis.show()

+----------+--------------------+------------------+--------------------------+------------------------+------------------+---------------------+----------------------+----------------------------+
|order_year|new_customer_revenue|new_customer_count|returning_customer_revenue|returning_customer_count|     total_revenue|returning_revenue_pct|avg_new_customer_value|avg_returning_customer_value|
+----------+--------------------+------------------+--------------------------+------------------------+------------------+---------------------+----------------------+----------------------------+
|      2010|  1890156.3099999991|              1130|                      NULL|                    NULL|              NULL|                 NULL|    1672.7046991150435|                        NULL|
|      2011|           912339.29|               714|        1009628.8299999993|                     499|1921968.1199999992|   52.530987350612236|    1277.7861204481794|          2023.3042685370726|
|      201

#### What is my annual total number of orders of products shipped?   

In [26]:
orders.select(
    "order_year", 
    "order_id",
    "customer_id"
).groupBy(
    "order_year"
).agg(
    countDistinct("order_id").alias("no_of_orders"),
    countDistinct("customer_id").alias("no_of_customers")
).withColumn(
    "orders_per_customer", 
    round(col("no_of_orders") / col("no_of_customers"), 2)
).orderBy(
    "order_year"
).show()

+----------+------------+---------------+-------------------+
|order_year|no_of_orders|no_of_customers|orders_per_customer|
+----------+------------+---------------+-------------------+
|      2010|        1365|           1130|               1.21|
|      2011|        1454|           1213|                1.2|
|      2012|        1487|           1232|               1.21|
|      2013|        2185|           1589|               1.38|
+----------+------------+---------------+-------------------+

